In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import xgboost

from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from statsmodels.graphics.tsaplots import plot_pacf
from xgboost import XGBRegressor
import lightgbm

In [2]:
all_csv = pd.read_csv('cleaned.csv.gz',
                 dtype = {
                     'store_nbr' : 'category',
                     'family' : 'category',
                     'sales': 'float',
                     'city': 'category',
                     'state': 'category',
                     'type': 'category',
                     'holiday_type': 'category',
                     'holiday_transferred': 'category'
                 },
                  parse_dates=['date'])
all_csv['date'] = pd.to_datetime(all_csv['date']).dt.to_period('D')

In [3]:
all = all_csv.copy()  # we can start experimenting from here without reloading the csv file

In [4]:
# this is for experimentation

filter_by_stores = None  # note: please use string here (unlike Mine.ipynb)
filter_by_family = None
filter_by_dates = None

#filter_by_stores = ['15']  # note: please use string here (unlike Mine.ipynb)
#filter_by_family = ['PRODUCE', 'AUTOMOBILE']
#filter_by_dates = '2014-06-05'

In [5]:
if filter_by_dates == None:
    train_start_date = '2013-01-01'
else:
    train_start_date = filter_by_dates
train_end_date = '2017-08-15'
test_start_date = '2017-08-16'
test_end_date = '2017-08-31'

In [6]:
if filter_by_family != None:
    all = all[all['family'].isin(filter_by_family)]
if filter_by_stores != None:
    all = all[all['store_nbr'].isin(filter_by_stores)]
if filter_by_dates != None:
    all = all[all['date'] >= filter_by_dates]

In [7]:
all['store_nbr'].unique()

['1', '10', '11', '12', '13', ..., '54', '6', '7', '8', '9']
Length: 54
Categories (54, object): ['1', '10', '11', '12', ..., '6', '7', '8', '9']

In [8]:
all.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3036528 entries, 0 to 3036527
Data columns (total 38 columns):
 #   Column               Dtype    
---  ------               -----    
 0   date                 period[D]
 1   store_nbr            category 
 2   family               category 
 3   sales                float64  
 4   onpromotion          int64    
 5   sales_lag_01         float64  
 6   sales_lag_02         float64  
 7   sales_lag_03         float64  
 8   sales_lag_04         float64  
 9   sales_lag_05         float64  
 10  sales_lag_06         float64  
 11  sales_lag_07         float64  
 12  sales_lag_08         float64  
 13  sales_lag_09         float64  
 14  sales_lag_10         float64  
 15  sales_lag_11         float64  
 16  sales_lag_12         float64  
 17  sales_lag_13         float64  
 18  sales_lag_14         float64  
 19  sales_lag_15         float64  
 20  sales_lag_16         float64  
 21  sales_lag_17         float64  
 22  sales_lag_18      

## One Hot Encoding

In [9]:
def one_hot_encode(df):
    ohe = OneHotEncoder()
    #ohe.fit_transform(df[column])
    return pd.get_dummies(data=df, columns=['store_nbr', 'family', 'city', 'state', 'type',
                                     'cluster', 'holiday_type', 'holiday_transferred', 'weekday'])    

In [10]:
all_ohe = one_hot_encode(all)
all_ohe = all_ohe.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))  # remove bad char in column names

X = all_ohe[all_ohe['date'] <= train_end_date]
X = X.drop(['sales'], axis=1)
y = all_ohe[['date', 'sales']][all_ohe['date'] <= train_end_date]
y.set_index('date', inplace=True)

X_test = all_ohe[all_ohe['date'] >= test_start_date]
X_test = X_test.drop(['sales'], axis=1)

X.drop('date', axis=1, inplace=True)
X_test.drop('date', axis=1, inplace=True)
y.set_index(X.index, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

## Experiment I -- Linear Regression

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [12]:
y_pred_train = lr.predict(X_train)
y_pred_train[y_pred_train < 0] = 0
y_pred_val = lr.predict(X_val)
y_pred_val[y_pred_val < 0] = 0

print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))

RMS log-error train:  0.16114945261294478
RMS log-error val:  0.16183534811749348


## Experiment II -- Lightgbm

In [13]:
lgb_params = {
    'boosting_type' : 'gbdt',  # gradient boosting decision tree
    'early_stopping_rounds': 200,
    'force_col_wise': True,
    'learning_rate': 0.1,
    'max_depth': 10,
    'metric': 'mse',  # mean square error
    'num_iterations': 5000,
    'num_leaves': 10,
    'random_state': 1,
    'verbose': 0
}

X_train_lgb = lightgbm.Dataset(data=X_train, label=y_train, feature_name='auto')
X_val_lgb = lightgbm.Dataset(data=X_val, label=y_val, reference=X_train_lgb, feature_name='auto')

In [14]:
lgb = lightgbm.train(
    params=lgb_params, 
    train_set=X_train_lgb,
    valid_sets=[X_train_lgb, X_val_lgb],
)

/usr/local/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[1]	training's l2: 4.90168	valid_1's l2: 4.91242
Training until validation scores don't improve for 200 rounds
[2]	training's l2: 4.02867	valid_1's l2: 4.0381
[3]	training's l2: 3.31896	valid_1's l2: 3.32735
[4]	training's l2: 2.74165	valid_1's l2: 2.74905
[5]	training's l2: 2.26981	valid_1's l2: 2.27645
[6]	training's l2: 1.88693	valid_1's l2: 1.89284
[7]	training's l2: 1.57317	valid_1's l2: 1.57847
[8]	training's l2: 1.31788	valid_1's l2: 1.32251
[9]	training's l2: 1.1096	valid_1's l2: 1.11375
[10]	training's l2: 0.93936	valid_1's l2: 0.943075
[11]	training's l2: 0.800775	valid_1's l2: 0.804217
[12]	training's l2: 0.688043	valid_1's l2: 0.691191
[13]	training's l2: 0.595532	valid_1's l2: 0.598386
[14]	training's l2: 0.520251	valid_1's l2: 0.522885
[15]	training's l2: 0.458727	valid_1's l2: 0.461112
[16]	training's l2: 0.40845	valid_1's l2: 0.410705
[17]	training's l2: 0.367455	valid_1's l2: 0.369544
[18]	training's l2: 0.333578	valid_1's l2: 0.335556
[19]	training's l2: 0.30593	valid

[158]	training's l2: 0.157119	valid_1's l2: 0.158281
[159]	training's l2: 0.157056	valid_1's l2: 0.158229
[160]	training's l2: 0.156985	valid_1's l2: 0.158155
[161]	training's l2: 0.156918	valid_1's l2: 0.15809
[162]	training's l2: 0.156849	valid_1's l2: 0.158023
[163]	training's l2: 0.156836	valid_1's l2: 0.158007
[164]	training's l2: 0.156763	valid_1's l2: 0.157934
[165]	training's l2: 0.156668	valid_1's l2: 0.15783
[166]	training's l2: 0.156613	valid_1's l2: 0.157785
[167]	training's l2: 0.156584	valid_1's l2: 0.157772
[168]	training's l2: 0.156533	valid_1's l2: 0.157742
[169]	training's l2: 0.156488	valid_1's l2: 0.157696
[170]	training's l2: 0.156409	valid_1's l2: 0.15762
[171]	training's l2: 0.156357	valid_1's l2: 0.157571
[172]	training's l2: 0.1562	valid_1's l2: 0.157418
[173]	training's l2: 0.156168	valid_1's l2: 0.157388
[174]	training's l2: 0.156128	valid_1's l2: 0.157357
[175]	training's l2: 0.156093	valid_1's l2: 0.157321
[176]	training's l2: 0.156035	valid_1's l2: 0.15726

[315]	training's l2: 0.149728	valid_1's l2: 0.151473
[316]	training's l2: 0.14972	valid_1's l2: 0.151466
[317]	training's l2: 0.149627	valid_1's l2: 0.151332
[318]	training's l2: 0.149594	valid_1's l2: 0.151307
[319]	training's l2: 0.149557	valid_1's l2: 0.151266
[320]	training's l2: 0.149536	valid_1's l2: 0.151253
[321]	training's l2: 0.149507	valid_1's l2: 0.151227
[322]	training's l2: 0.149473	valid_1's l2: 0.151198
[323]	training's l2: 0.14946	valid_1's l2: 0.15119
[324]	training's l2: 0.149437	valid_1's l2: 0.151172
[325]	training's l2: 0.149426	valid_1's l2: 0.151161
[326]	training's l2: 0.149417	valid_1's l2: 0.151153
[327]	training's l2: 0.149406	valid_1's l2: 0.151142
[328]	training's l2: 0.149394	valid_1's l2: 0.151131
[329]	training's l2: 0.149383	valid_1's l2: 0.151122
[330]	training's l2: 0.149372	valid_1's l2: 0.151113
[331]	training's l2: 0.149328	valid_1's l2: 0.1511
[332]	training's l2: 0.149248	valid_1's l2: 0.151023
[333]	training's l2: 0.149224	valid_1's l2: 0.15099

[471]	training's l2: 0.145522	valid_1's l2: 0.14784
[472]	training's l2: 0.145514	valid_1's l2: 0.147834
[473]	training's l2: 0.145508	valid_1's l2: 0.147828
[474]	training's l2: 0.1455	valid_1's l2: 0.147822
[475]	training's l2: 0.145492	valid_1's l2: 0.147814
[476]	training's l2: 0.145473	valid_1's l2: 0.147796
[477]	training's l2: 0.145418	valid_1's l2: 0.147753
[478]	training's l2: 0.145406	valid_1's l2: 0.147749
[479]	training's l2: 0.145378	valid_1's l2: 0.147727
[480]	training's l2: 0.145365	valid_1's l2: 0.147713
[481]	training's l2: 0.145338	valid_1's l2: 0.147696
[482]	training's l2: 0.14532	valid_1's l2: 0.147678
[483]	training's l2: 0.14531	valid_1's l2: 0.147668
[484]	training's l2: 0.145299	valid_1's l2: 0.14766
[485]	training's l2: 0.145288	valid_1's l2: 0.147651
[486]	training's l2: 0.145277	valid_1's l2: 0.14764
[487]	training's l2: 0.145258	valid_1's l2: 0.147624
[488]	training's l2: 0.145236	valid_1's l2: 0.147605
[489]	training's l2: 0.145217	valid_1's l2: 0.147587


...

[4352]	training's l2: 0.12032	valid_1's l2: 0.130485
[4353]	training's l2: 0.120316	valid_1's l2: 0.130483
[4354]	training's l2: 0.120313	valid_1's l2: 0.13048
[4355]	training's l2: 0.120312	valid_1's l2: 0.130482
[4356]	training's l2: 0.120309	valid_1's l2: 0.130481
[4357]	training's l2: 0.120307	valid_1's l2: 0.130482
[4358]	training's l2: 0.120305	valid_1's l2: 0.130481
[4359]	training's l2: 0.120302	valid_1's l2: 0.13048
[4360]	training's l2: 0.120299	valid_1's l2: 0.130477
[4361]	training's l2: 0.120296	valid_1's l2: 0.130476
[4362]	training's l2: 0.120294	valid_1's l2: 0.130474
[4363]	training's l2: 0.120293	valid_1's l2: 0.130474
[4364]	training's l2: 0.120292	valid_1's l2: 0.130473
[4365]	training's l2: 0.12029	valid_1's l2: 0.130472
[4366]	training's l2: 0.120283	valid_1's l2: 0.130465
[4367]	training's l2: 0.12028	valid_1's l2: 0.130462
[4368]	training's l2: 0.120277	valid_1's l2: 0.13046
[4369]	training's l2: 0.120274	valid_1's l2: 0.130459
[4370]	training's l2: 0.12027	vali

[4506]	training's l2: 0.119888	valid_1's l2: 0.13021
[4507]	training's l2: 0.119885	valid_1's l2: 0.130209
[4508]	training's l2: 0.119882	valid_1's l2: 0.130207
[4509]	training's l2: 0.119879	valid_1's l2: 0.130206
[4510]	training's l2: 0.119877	valid_1's l2: 0.130206
[4511]	training's l2: 0.119875	valid_1's l2: 0.130205
[4512]	training's l2: 0.119873	valid_1's l2: 0.130204
[4513]	training's l2: 0.11987	valid_1's l2: 0.130202
[4514]	training's l2: 0.119867	valid_1's l2: 0.130201
[4515]	training's l2: 0.119864	valid_1's l2: 0.1302
[4516]	training's l2: 0.119861	valid_1's l2: 0.130199
[4517]	training's l2: 0.119859	valid_1's l2: 0.130198
[4518]	training's l2: 0.119857	valid_1's l2: 0.130197
[4519]	training's l2: 0.119854	valid_1's l2: 0.130195
[4520]	training's l2: 0.119852	valid_1's l2: 0.130194
[4521]	training's l2: 0.119849	valid_1's l2: 0.13019
[4522]	training's l2: 0.119847	valid_1's l2: 0.130188
[4523]	training's l2: 0.119836	valid_1's l2: 0.130173
[4524]	training's l2: 0.119833	va

[4662]	training's l2: 0.119428	valid_1's l2: 0.129951
[4663]	training's l2: 0.119426	valid_1's l2: 0.12995
[4664]	training's l2: 0.119421	valid_1's l2: 0.129944
[4665]	training's l2: 0.119418	valid_1's l2: 0.129942
[4666]	training's l2: 0.119413	valid_1's l2: 0.129937
[4667]	training's l2: 0.119409	valid_1's l2: 0.129936
[4668]	training's l2: 0.119404	valid_1's l2: 0.129933
[4669]	training's l2: 0.119399	valid_1's l2: 0.129929
[4670]	training's l2: 0.11939	valid_1's l2: 0.129917
[4671]	training's l2: 0.119387	valid_1's l2: 0.129916
[4672]	training's l2: 0.119383	valid_1's l2: 0.129915
[4673]	training's l2: 0.11938	valid_1's l2: 0.129911
[4674]	training's l2: 0.119377	valid_1's l2: 0.12991
[4675]	training's l2: 0.119373	valid_1's l2: 0.129911
[4676]	training's l2: 0.11937	valid_1's l2: 0.12991
[4677]	training's l2: 0.119367	valid_1's l2: 0.129907
[4678]	training's l2: 0.119364	valid_1's l2: 0.129906
[4679]	training's l2: 0.119361	valid_1's l2: 0.129905
[4680]	training's l2: 0.11936	vali

[4818]	training's l2: 0.118987	valid_1's l2: 0.12969
[4819]	training's l2: 0.118985	valid_1's l2: 0.12969
[4820]	training's l2: 0.118983	valid_1's l2: 0.129689
[4821]	training's l2: 0.118981	valid_1's l2: 0.129692
[4822]	training's l2: 0.118979	valid_1's l2: 0.129691
[4823]	training's l2: 0.118977	valid_1's l2: 0.12969
[4824]	training's l2: 0.118975	valid_1's l2: 0.12969
[4825]	training's l2: 0.118971	valid_1's l2: 0.129687
[4826]	training's l2: 0.118968	valid_1's l2: 0.129684
[4827]	training's l2: 0.118964	valid_1's l2: 0.129681
[4828]	training's l2: 0.11896	valid_1's l2: 0.129682
[4829]	training's l2: 0.118958	valid_1's l2: 0.129683
[4830]	training's l2: 0.118956	valid_1's l2: 0.12968
[4831]	training's l2: 0.118953	valid_1's l2: 0.129678
[4832]	training's l2: 0.11895	valid_1's l2: 0.129676
[4833]	training's l2: 0.118948	valid_1's l2: 0.129675
[4834]	training's l2: 0.118945	valid_1's l2: 0.129672
[4835]	training's l2: 0.118944	valid_1's l2: 0.129672
[4836]	training's l2: 0.118938	vali

[4972]	training's l2: 0.118563	valid_1's l2: 0.129439
[4973]	training's l2: 0.118562	valid_1's l2: 0.129439
[4974]	training's l2: 0.118557	valid_1's l2: 0.129437
[4975]	training's l2: 0.118555	valid_1's l2: 0.129436
[4976]	training's l2: 0.118553	valid_1's l2: 0.129434
[4977]	training's l2: 0.11855	valid_1's l2: 0.129432
[4978]	training's l2: 0.118548	valid_1's l2: 0.129431
[4979]	training's l2: 0.118546	valid_1's l2: 0.129429
[4980]	training's l2: 0.118543	valid_1's l2: 0.129426
[4981]	training's l2: 0.118541	valid_1's l2: 0.129425
[4982]	training's l2: 0.118539	valid_1's l2: 0.129423
[4983]	training's l2: 0.118533	valid_1's l2: 0.129423
[4984]	training's l2: 0.11853	valid_1's l2: 0.12942
[4985]	training's l2: 0.118526	valid_1's l2: 0.129417
[4986]	training's l2: 0.118524	valid_1's l2: 0.129416
[4987]	training's l2: 0.118522	valid_1's l2: 0.129416
[4988]	training's l2: 0.118518	valid_1's l2: 0.129413
[4989]	training's l2: 0.118516	valid_1's l2: 0.129412
[4990]	training's l2: 0.118513	

In [15]:
y_pred_train = lgb.predict(X_train, num_iteration=lgb.best_iteration)
y_pred_train[y_pred_train < 0] = 0
y_pred_val = lgb.predict(X_val, num_iteration=lgb.best_iteration)
y_pred_val[y_pred_val < 0] = 0

print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))

RMS log-error train:  0.1387609500374927
RMS log-error val:  0.14257049755882722


## Experiment III -- Random Forest (Too Slow)

In [16]:
# random_forest = RandomForestRegressor(random_state=1)
# random_forest.fit(X_train, y_train)

In [17]:
# y_pred_train = random_forest.predict(X_train)
# y_pred_train[y_pred_train < 0] = 0
# y_pred_val = random_forest.predict(X_val)
# y_pred_val[y_pred_val < 0] = 0
# 
# print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
# print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))

## Experiment IV -- XGBoost

In [18]:
xgb = xgboost.XGBRegressor(n_estimators=100, early_stopping_rounds=50, learning_rate=0.3)
xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        verbose=True)

[0]	validation_0-rmse:2.71987	validation_1-rmse:2.72299
[1]	validation_0-rmse:1.93035	validation_1-rmse:1.93278
[2]	validation_0-rmse:1.38620	validation_1-rmse:1.38824
[3]	validation_0-rmse:1.01648	validation_1-rmse:1.01831
[4]	validation_0-rmse:0.77172	validation_1-rmse:0.77343
[5]	validation_0-rmse:0.61584	validation_1-rmse:0.61747
[6]	validation_0-rmse:0.52174	validation_1-rmse:0.52354
[7]	validation_0-rmse:0.46771	validation_1-rmse:0.46957
[8]	validation_0-rmse:0.43778	validation_1-rmse:0.43960
[9]	validation_0-rmse:0.42168	validation_1-rmse:0.42367
[10]	validation_0-rmse:0.41267	validation_1-rmse:0.41490
[11]	validation_0-rmse:0.40735	validation_1-rmse:0.40977
[12]	validation_0-rmse:0.40454	validation_1-rmse:0.40705
[13]	validation_0-rmse:0.40260	validation_1-rmse:0.40516
[14]	validation_0-rmse:0.40071	validation_1-rmse:0.40338
[15]	validation_0-rmse:0.39936	validation_1-rmse:0.40220
[16]	validation_0-rmse:0.39789	validation_1-rmse:0.40095
[17]	validation_0-rmse:0.39717	validation

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=50, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.3, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [19]:
y_pred_train = xgb.predict(X_train)
y_pred_train[y_pred_train < 0] = 0
y_pred_val = xgb.predict(X_val)
y_pred_val[y_pred_val < 0] = 0

print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))

RMS log-error train:  0.14580213414791765
RMS log-error val:  0.148140172363345


## Test (Moment of Truth)

In [20]:
def main_predict(model, X_test):
    X_test_mod = X_test.copy()
    output = np.array([])
    start_day, end_day = X_test['day_of_month'].min(), X_test['day_of_month'].max()
        # we lost the dates, but we still have day_of_month, which is good enough for our experiment
        
    for day in range(start_day, end_day + 1):
        pred = model.predict(X_test_mod[X_test_mod['day_of_month'] == day])
        pred[pred < 0] = 0
        print(pred)
        output = np.concatenate([output, pred], axis=0)
        for future in range(day + 1, end_day + 1):
            X_test_mod.loc[X_test_mod[X_test_mod['day_of_month'] == future].index,
                           f'sales_lag_{(future - day):02d}'] = pred
            # fill out future values now that this sales figure is available
            
    return output

In [22]:
y_pred_test = main_predict(lgb, X_test)

[1.71110834 0.08379029 1.81252702 ... 7.39744987 4.69015147 2.72674146]
[1.66971891 0.16332013 1.62072741 ... 7.1026506  4.75359562 2.74782741]
[1.65037821 0.25248781 1.59161123 ... 7.21360495 4.56534682 2.71157835]
[1.69302112 0.26195894 1.64988892 ... 7.36286941 4.90834782 3.03219422]
[1.33943444 0.22488158 1.25700134 ... 7.50540871 5.28738142 3.10238349]
[1.80396964 0.45999522 1.7878795  ... 7.28954287 5.00519199 2.79132473]
[1.6229337  0.47176165 1.51336683 ... 7.75689353 4.862333   2.73353559]
[1.80236967 0.49506213 1.74457141 ... 7.25184884 4.55856768 2.65561607]
[1.58855556 0.45262751 1.55163497 ... 7.08625759 4.65245023 2.6803464 ]
[1.67862865 0.51737878 1.52286322 ... 7.19894014 4.47835593 2.69521436]
[1.72424265 0.49387913 1.68222807 ... 7.38367885 4.85324375 3.19632852]
[1.31565798 0.37330859 1.32004018 ... 7.4995543  5.25334404 3.12507901]
[1.71209913 0.614586   1.75465548 ... 7.16633549 5.03806483 2.67560275]
[1.63097807 0.63489693 1.6891822  ... 7.74792852 4.76284994 2.70

In [25]:
delta_index = 3008016 - 3000888  # we inserted 4 Christmas days, 4 x 54 x 33 = 7128, which is the difference
submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
submission.to_csv('submission.csv', index=False)